# Lab02

Unicode input: type "\\varphi" to obtain φ, "\\rho" for ρ, and similarly for other mathematical characters.

# Imports and utility functions

In [1]:
{-# LANGUAGE UnicodeSyntax, TypeSynonymInstances, FlexibleInstances #-}

import Data.List
import Control.Monad
import Control.Monad.State
import Test.QuickCheck
import System.IO.Unsafe

-- updating a function
update :: Eq a => (a -> b) -> a -> b -> a -> b
update ρ x v = \y -> if x == y then v else ρ y

-- useful for debugging
debug :: Show a => String -> a -> a
debug str x = seq (unsafePerformIO $ do putStr "<"; putStr str; putStr ": "; print x; putStr ">") x

todo = undefined

# Syntax

We define an inductive type for formulas of propositional logic.

In [1]:
-- Variable names are just strings
type PropName = String

-- generation of fresh variable names
fresh :: [PropName] -> PropName
fresh vars = head $ filter (not . (`elem` vars)) $ map (("p"++) . show) [0..]

data Formula =
      T
    | F
    | Prop PropName -- atomic formulas
    | Not Formula
    | And Formula Formula
    | Or Formula Formula
    | Implies Formula Formula
    | Iff Formula Formula
    deriving (Eq, Ord, Show)
    
p, q, r, s, t :: Formula

p = Prop "p"
q = Prop "q"
r = Prop "r"
s = Prop "s"
t = Prop "t"

We define infix versions of some formula constructors:

In [1]:
infixr 8 /\, ∧
(/\) = And
(∧) = And

infixr 5 \/, ∨, ==>
(\/) = Or
(∨) = Or -- input with "\or"
(==>) = Implies

infixr 4 <==>, ⇔
(<==>) = Iff
(⇔) = Iff -- input with "\lr"

Random generation of propositional formulas (for testing):

In [1]:
instance Arbitrary Formula where
    arbitrary = sized f where
      
      f 0 = oneof $ map return $ [p, q, r, s, t] ++ [T, F]

      f size = frequency [
        (1, liftM Not (f (size - 1))),
        (4, do
              size' <- choose (0, size - 1)
              conn <- oneof $ map return [And, Or, Implies, Iff]
              left <- f size'
              right <- f $ size - size' - 1
              return $ conn left right)
        ]
        
    shrink (Not φ) = [φ]
    shrink (Or φ ψ) = [φ, ψ]
    shrink (And φ ψ) = [φ, ψ]
    shrink (Implies φ ψ) = [φ, ψ]
    shrink (Iff φ ψ) = [φ, ψ]
    shrink _ = []

Line 7: Use fmap
Found:
liftM
Why not:
fmap

# In the previous episode

Semantics of propositional logic:

In [1]:
type Valuation = PropName -> Bool

eval :: Formula -> Valuation -> Bool
eval T _ = True
eval F _ = False
eval (Prop p) ρ = ρ p
eval (Not φ) ρ = not (eval φ ρ)
eval (And φ ψ) ρ = (eval φ ρ) && (eval ψ ρ)
eval (Or φ ψ) ρ = (eval φ ρ) || (eval ψ ρ)
eval (Implies φ ψ) ρ = not (eval φ ρ) || (eval ψ ρ)
eval (Iff φ ψ) ρ = (eval φ ρ == eval ψ ρ)

Variables appearing in the formula:

In [1]:
variables :: Formula -> [PropName]
variables = nub . go where
  go T = []
  go F = []
  go (Prop p) = [p]
  go (Not φ) = go φ
  go (And φ ψ) = go φ ++ go ψ
  go (Or φ ψ) = go φ ++ go ψ
  go (Implies φ ψ) = go φ ++ go ψ
  go (Iff φ ψ) = go φ ++ go ψ

All truth assignments on a given list of variables:

In [1]:
valuations :: [PropName] -> [Valuation]
valuations [] = [undefined]
valuations (x : xs) = concat [[update ϱ x True, update ϱ x False] | ϱ <- valuations xs]

A SAT solver based on truth-tables:

In [1]:
type SATSolver = Formula -> Bool

satisfiable :: SATSolver
satisfiable φ = or [eval φ ϱ | ϱ <- valuations (variables φ)]

A tautology checker:

In [1]:
tautology :: Formula -> Bool
tautology φ = and [eval φ ρ | ρ <- valuations (variables φ)]

Negation normal form:

In [1]:
nnf :: Formula -> Formula
nnf T = T
nnf F = F
nnf r@(Prop p) = r
nnf (Not phi) = case nnf phi of
  T -> F
  F -> T
  Not phi' -> phi'
  Or phi' psi' -> And (nnf (Not phi')) (nnf (Not psi'))
  And phi' psi' -> Or (nnf (Not phi')) (nnf (Not psi'))
  phi' -> Not phi'
nnf (And phi psi) = And (nnf phi) (nnf psi)
nnf (Or phi psi) = Or (nnf phi) (nnf psi)
nnf (Implies phi psi) = Or (nnf (Not phi)) (nnf psi)
nnf (Iff phi psi) = nnf ((phi ==> psi) /\ (psi ==> phi))

# More normal forms

## Conjunctive normal form (CNF)

A *literal* is either  a propositional variable, or the negation of a propositional variable:

In [1]:
data Literal = Pos PropName | Neg PropName deriving (Eq, Show, Ord)

literal2var :: Literal -> PropName
literal2var (Pos p) = p
literal2var (Neg p) = p

opposite :: Literal -> Literal
opposite (Pos p) = Neg p
opposite (Neg p) = Pos p

A *clause* is a disjunction of literals. A formula of propositional logic is in *conjunctive normal form* (CNF) if it is a conjunction of clauses. It is customary to represent CNF formulas as lists of lists of literals:

In [1]:
type CNFClause = [Literal]
type CNF = [CNFClause]

cnf2formula :: CNF -> Formula
cnf2formula [] = T
cnf2formula lss = foldr1 And (map go lss) where
  go [] = F
  go ls = foldr1 Or (map go2 ls)
  go2 (Pos p) = Prop p
  go2 (Neg p) = Not (Prop p)
  
positive_literals :: CNFClause -> [PropName]
positive_literals ls = nub [p | Pos p <- ls]

negative_literals :: CNFClause -> [PropName]
negative_literals ls = nub [p | Neg p <- ls]

literals :: [Literal] -> [PropName]
literals ls = nub $ positive_literals ls ++ negative_literals ls

Converting an arbitrary formula to a *logically equivalent* one in CNF is entirely dual to the DNF translation from the previous lab:

In [1]:
cnf :: Formula -> CNF
cnf = go . nnf where
  go T = []
  go F = [[]]
  go (Prop p) = [[Pos p]]
  go (Not (Prop p)) = [[Neg p]]
  go (φ `And` ψ) = go φ ++ go ψ
  go (φ `Or` ψ) = [as ++ bs | as <- go φ, bs <- go ψ]

Tests:

In [1]:
test_cnf :: Formula -> Bool
test_cnf φ = tautology $ φ ⇔ (cnf2formula (cnf φ))

quickCheckWith (stdArgs {maxSize = 18}) test_cnf

+++ OK, passed 100 tests.

## Equisatisfiable conjunctive normal form (ECNF) (EXERCISE)

Two propositional formulas are *equisatisfiable* if they are either both satisfiable, or both unsatisfiable:

In [1]:
equi_satisfiable :: Formula -> Formula -> Bool
equi_satisfiable φ ψ = satisfiable φ == satisfiable ψ

Note that 1) if two formulas are logically equivalent, then they are also equisatisfiable, and 2) the other way around is false: For example, $\varphi \equiv p$ and $\psi \equiv \top$ are both satisfiable, and thus they are equisatisfiable, however they clearly are not logically equivalent.

Write a function transforming a propositional formula into an equisatisfiable CNF formula of polynomial size. Fresh variable names can be generated according to the helper function "fresh" defined at the beginning.

In [1]:
helper :: (Formula -> Formula -> Formula) -> Formula -> Formula -> [PropName] -> ([PropName], Formula, CNF)
helper op phi psi vars = (vars'', Prop v_i, cnf_i)
  where
    v_i = fresh vars
    (vars', v_j, cnf_j) = tseitin phi (v_i:vars)
    (vars'', v_k, cnf_k) = tseitin psi vars'
    cnf_i = cnf_j ++ cnf_k ++ cnf (Iff (Prop v_i) (op v_j v_k))

tseitin :: Formula -> [PropName] -> ([PropName], Formula, CNF)
tseitin T vars = (vars, T, [])
tseitin F vars = (vars, F, [])
tseitin (Prop p) vars = (vars, Prop p, [])
tseitin (Not phi) vars = let (vars', v_i, cnf_i) = tseitin phi vars in (vars', Not v_i, cnf_i)
tseitin (And phi psi) vars = helper And phi psi vars
tseitin (Or phi psi) vars = helper Or phi psi vars
tseitin (Implies phi psi) vars = helper Implies phi psi vars
tseitin (Iff phi psi) vars = helper Iff phi psi vars

ecnf :: Formula -> CNF
ecnf f = let (_, v, cnf') = tseitin f [] in cnf v ++ cnf'

ecnf (T ∧ F ∨ T)

[[Pos "p0"],[Neg "p1"],[Neg "p1",Pos "p0"],[Pos "p0"]]

Tests:

In [1]:
prop_ecnf :: Formula -> Bool
prop_ecnf phi = equi_satisfiable phi (cnf2formula $ ecnf phi)

quickCheckWith (stdArgs {maxSize = 10}) prop_ecnf

+++ OK, passed 100 tests.

# Davis-Putnam SAT solver

## Tautological clauses (EXERCISE)

A SAT clause is *tautological* if it contains some literal both positively and negatively. Write a function that removes tautological clauses:

In [1]:
is_tautological :: CNFClause -> Bool
is_tautological clause = any (\p -> p `elem` negLit) posLit
  where
    posLit = positive_literals clause
    negLit = negative_literals clause

remove_tautologies :: CNF -> CNF
remove_tautologies = filter (not . is_tautological)

## One literal rule (EXERCISE)

A *one-literal* clause is a clause containing only one literal $\ell$. If a CNF contains a one-literal clause $\ell$*,* say a positive literal $\ell = p$, then $p$ must necessarily be true in any satisfying assignment (if any exists). Consequently, we can remove all clauses containing $p$ positively, and we can remove all occurrences of the opposite literal $\neg p$ from all the other clauses. The same idea can be (dually) applied if $\ell = \neg p$ is a one-literal clause.

Write a function that transforms a given CNF into an equisatisfiable one without one-literal clauses:

In [1]:
remove_one_literals :: CNF -> CNF
remove_one_literals cnf = foldl helper cnf single_lits
  where
    single_lits = concat $ filter (\clause -> length clause == 1) (map nub cnf)
    
    helper :: CNF -> Literal -> CNF
    helper clauses l = map (filter (\lit -> lit /= opposite l)) clauses'
      where
        clauses' = filter (notElem l) clauses

one_literal :: CNF -> CNF
one_literal cnf = if cnf == cnf' then cnf else one_literal cnf'
  where
    cnf' = remove_one_literals cnf

one_literal
    [[Pos "p"], [Pos "p", Pos "q", Pos "p", Pos "r"], [Neg "q", Pos "r", Neg "p", Neg "r", Neg "p"], [Neg "q", Neg "p"], [Pos "q", Pos "r", Pos "s"], [Neg "p", Pos "p"]]

[[Pos "r",Pos "s"]]

Note: Removing a proposition $p$ in this way can generate new one-literal clauses.

Tests:

In [1]:
prop_one_literal :: Bool
prop_one_literal =
  one_literal
    [[Pos "p"], [Pos "p", Pos "q", Pos "p", Pos "r"], [Neg "q", Pos "r", Neg "p", Neg "r", Neg "p"], [Neg "q", Neg "p"], [Pos "q", Pos "r", Pos "s"], [Neg "p", Pos "p"]] ==
    [[Pos "r",Pos "s"]] &&
  one_literal
    [[Pos "p"],[Pos "p1"],[Neg "p",Pos "q"],[Pos "p1",Pos "p0"],[Pos "q",Neg "p0",Pos "p1"],[Neg "p0",Pos "s"],[Neg "q0",Neg "p"],[Neg "s",Neg "p",Pos "p0"]] ==
    [[Neg "p0",Pos "s"],[Neg "s",Pos "p0"]] &&
  one_literal
    [[Pos "q"],[Pos "p0"],[Neg "p0",Pos "s"],[Neg "p0"]] ==
    [[]]
    
quickCheck prop_one_literal

+++ OK, passed 1 test.

## Affirmative-negative rule (EXERCISE)

If a literal appears either only positively, or only negatively, then all clauses where it occurs can be removed, preserving satisfiability. Write a function that removes all literals which appear only positively, or only negatively:

In [1]:
affirmative_negative :: CNF -> CNF
affirmative_negative cnf = if single_lits == [] then cnf else affirmative_negative $ helper cnf
  where
    all_lits = nub $ concat cnf
    single_lits = filter (\lit -> opposite lit `notElem` all_lits) all_lits
    
    helper :: CNF -> CNF
    helper clauses = foldl (\acc lit -> filter (notElem lit) acc) clauses single_lits

Tests:

In [1]:
prop_affirmative_negative :: Bool
prop_affirmative_negative =
  affirmative_negative
    [[Neg "p2",Pos "p"],[Neg "p2",Pos "p1"],[Neg "p",Neg "p1",Neg "p2"],[Neg "p1",Pos "q"],[Neg "p1",Pos "p0"],[Neg "q",Neg "p0",Pos "p1"],[Neg "p0",Pos "s"],[Neg "p0",Neg "p"],[Neg "s",Pos "p",Pos "p0"]] ==
    [[Neg "p1",Pos "q"],[Neg "p1",Pos "p0"],[Neg "q",Neg "p0",Pos "p1"],[Neg "p0",Pos "s"],[Neg "p0",Neg "p"],[Neg "s",Pos "p",Pos "p0"]] &&
  affirmative_negative
    [[Pos "p", Pos "q"], [Pos "p", Neg "q"]] ==
    []
    
quickCheck prop_affirmative_negative

+++ OK, passed 1 test.

## Resolution (EXERCISE)

After applying all the previous rules, we know that 1) every variable appears positively and negatively, 2) no variable appears both positively and negatively in the same clause, 3) there is at least one clause, and 4) all clauses are nonempty.

We can thus select a variable that appears both positively and negatively (necessarily from different clauses) and perform resolution in all possible ways and remove this variable altogether. Write a function that removes such a variable:

In [1]:
resolution :: CNF -> CNF
resolution cnf = [helper phi psi | phi <- posCls, psi <- negCls] ++ rest
  where
    lit = head $ head cnf
    opp = opposite lit
    posCls = filter (elem lit) cnf
    negCls = filter (elem opp) cnf
    rest = filter (notElem lit) $ filter (notElem opp) cnf

    helper :: CNFClause -> CNFClause -> CNFClause
    helper phi psi = phi' ++ psi'
      where
        phi' = filter (/= lit) phi
        psi' = filter (/= opp) psi

resolution [[Pos "p", Pos "q"],[Neg "p", Neg "q"]]

[[Pos "q",Neg "q"]]

Tests:

In [1]:
prop_resolution :: Bool
prop_resolution = resolution [[Pos "p", Pos "q"],[Neg "p", Neg "q"]] == [[Pos "q", Neg "q"]]

quickCheck prop_resolution

+++ OK, passed 1 test.

## Davis-Putnam algorithm (EXERCISE)

By combining the previous steps we obtain the Davis-Putnam satisfiability algorithm: Convert the input formula to ECNF, and then:

1. If the CNF is empty, then it is satisfiable.
2. If the CNF contains  an empty clause, then it is not satisfiable.
3. Remove all tautological clauses.
4. Apply the one-literal rule until it can no longer be applied.
5. Apply the affirmative-negative rule until it can no longer be applied.
6. Only when 3., 4., and 5. above can no longer be applied, apply resolution, and start again from the beginning.

In [1]:
dp :: CNF -> Bool
dp cnf = if cnf /= cnf' then dp cnf' else
  if cnf == [] then True
  else if [] `elem` cnf then False
  else dp $ resolution cnf'
  where
    cnf' = affirmative_negative $ one_literal $ remove_tautologies cnf

sat_DP :: SATSolver
sat_DP form = dp cnf where
  cnf = ecnf form

Tests:

In [1]:
prop_DP :: Formula -> Bool
prop_DP φ = sat_DP φ == satisfiable φ

quickCheckWith (stdArgs {maxSize = 10}) prop_DP

+++ OK, passed 100 tests.

In [1]:
%%sh
stack install QuickCheck